In [ ]:
import pandas as pd
import os
import sys
from tqdm import tqdm
import gc
import math

# Chunking & Filtering

The datasets are too large to import directly into Google Sheets. I'm using Python to import the taxi files via chunks, filtering the chunks, and then concantinating the results to create datasets of rides between 4/14/2014 - 4/20/2014.

The progress bar was added for reference, as this process took several minutes. There were also issues with low memory that needed to be monitored. Steps were taken below to reduce memory usage.

### Yellow and Green taxi

In [149]:
# Code adapted from: https://stackoverflow.com/a/66519332
# Troubleshooting help courtesy of Alex Baransky

# For counting rows
def row_count(filename):
    with open(filename) as f:
        for i, l in enumerate(f):
            pass
    return i

filename = 'green_taxi.csv'
# filename = 'yellow_taxi.csv'

use_cols = ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'trip_distance', 
            'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
           'tip_amount', 'total_amount']
# dtype_dict = {'vendor_id': 'object', 'trip_distance': 'float', 'pickup_longitude': 'float', 'pickup_latitude': 'float',
#               'dropoff_longitude': 'float', 'dropoff_latitude': 'float', 'tip_amount': 'float', 'total_amount': 'float'}

start_date = pd.to_datetime('04/14/2014 12:00:00 AM')
end_date = pd.to_datetime('04/20/2014 11:59:59 PM')

num_rows = row_count(filename) # Find total rows in data
# num_rows = 165114361 # Found it by running the above function, takes long!
chunk_size = 1e5
temp = pd.read_csv(filename, nrows=0)
temp.columns = temp.columns.str.lower().str.replace('vendorid', 'vendor_id')

df_list = [temp[:0]] # We want to make a list of chunk dfs which we will collapse (concat) later, this is just an empty
                        # df for the colnames
    
t = math.ceil(num_rows/chunk_size) # This isn't really important, something to do with progress bar

with tqdm(total = t, file = sys.stdout) as pbar: # Some more progress bar stuff?

    # Below is iterating over the chunks
    for i, chunk in enumerate(pd.read_csv(filename, chunksize=chunk_size, low_memory=False,
                                          usecols = use_cols, names = temp.columns)): # dtype = dtype_dict,

        chunk['pickup_datetime'] = pd.to_datetime(chunk['pickup_datetime'], errors='coerce', format = '%m/%d/%Y %I:%M:%S %p')
        
        # We wrote the bottom 2 lines, creating mask and filtering the chunk
        mask = (chunk['pickup_datetime'] > start_date) & (chunk['pickup_datetime'] < end_date)
        
        if mask.any():
            df_list.append(chunk[mask][use_cols]) # Add the filtered chunk to the df_list
            
        pbar.set_description('Importing: %d' % (1 + i)) # More progress bar stuff?
        pbar.update(1) # ditto
        

data = temp[:0].append(df_list) # This is actually the concat step, but we could have done the below too

# data = pd.concat(df_list)

del df_list # Free up memory 

data.to_csv(f'filtered_{filename}')

Importing: 159: 100%|████████████████████████████████████████████████████████████████| 159/159 [01:11<00:00,  2.22it/s]


### Uber data

This file was smaller and did not have to be chunked during import, only filtered.

In [154]:
# Filtering the Uber data

uber = pd.read_csv('uber_apr2014.csv')

uber['Date/Time'] = pd.to_datetime(uber['Date/Time'])
uber_clean = uber[(uber['Date/Time'] > start_date) & (uber['Date/Time'] < end_date)]

In [156]:
uber_clean.to_csv('filtered_uber.csv')